Shape-aware list ADT.

In [1]:
sealed abstract class List[A]
case class Nil[A]() extends List[A]
case class Cons[A, T <: List[A]](head: A, tail: T) extends List[A]

defined class List
defined class Nil
defined class Cons

In [2]:
import scala.io.StdIn.readLine

import scala.io.StdIn.readLine

Type-level function that receives the shape of a list 
and returns an instance of that precise shape obtained from the standard input 
(provided the list typed matches the specified shape).



In [3]:
// Type-level signature

trait ReadList[L <: List[String]]{
    def apply(): Option[L]
}

object ReadList{
    
    def apply[L <: List[String]](implicit R: ReadList[L]): Option[L] = 
        R.apply
    
    // Type-level definition
    
    implicit object NilReadList extends ReadList[Nil[String]]{
        def apply(): Option[Nil[String]] = 
            if (readLine == "") Some(Nil())
            else None
    }
    
    implicit def ConsReadList[T <: List[String]](implicit T: ReadList[T]) = new ReadList[Cons[String, T]]{
        def apply(): Option[Cons[String, T]] = {
            val head = readLine
            if (head == "") None
            else T.apply.map(Cons(head, _))
        }
    }
}
    

defined trait ReadList
defined object ReadList

Read a list of two elements from the standard input (the actual `ReadList` function is constructed at compile-time):

In [4]:
ReadList[Cons[String, Cons[String, Nil[String]]]]

1
2
3


res3: Option[Cons[String, Cons[String, Nil[String]]]] = None

In [5]:
ReadList[Cons[String, Cons[String, Nil[String]]]]

1
2



res4: Option[Cons[String, Cons[String, Nil[String]]]] = Some(
  Cons("1", Cons("2", Nil()))
)

This function reads a generic list from the standard input (the shape is lost):


In [6]:
def gen: List[String] = {
    val head: String = readLine
    if (head == "") Nil[String]()
    else Cons(head, gen)
}

defined function gen

In [7]:
val l: List[String] = gen

1
2



l: List[String] = Cons("1", Cons("2", Nil()))

But we may reconstruct the type-level function given a generic list:

In [8]:
 def readListOfShape(l: List[String]): ReadList[_ <: List[String]] = 
    l match {
        case Nil() => 
            ReadList.NilReadList
        case Cons(head, tail: List[String]) => 
            ReadList.ConsReadList(readListOfShape(tail))
    }  


defined function readListOfShape

The result of the previous function is an existential type, but we can read now a new list from the standard input of the same exact shape that the one read before:

In [9]:
readListOfShape(l)()

1
2
3


res8: Option[List[String]] = None

In [11]:
readListOfShape(l)()

1
2



res10: Option[List[String]] = Some(Cons("1", Cons("2", Nil())))